In [1]:
#automatically reload modules when they have changed
%reload_ext autoreload
from pathlib import Path
import random
import os

from yaml_config_wrapper import Configuration
from RLcraft import MalmoMazeEnv

In [2]:
yml_path = 'configs/mazes.yml'
# Load YML config file
c = Configuration(config_src=yml_path)
# Load configs from config class
c_general = c.get_config('general')[0]
c_tuner = c.get_config('tuner')[0]
# Load the values from the config
run_config = c_tuner['config']
env_config = run_config['env_config']
c_general = c_general['config']

Configuration file loaded successfully from path: /Users/gkos/Insync/Gdrive/Projects/UTK/Minecraft-AI/configs/mazes.yml
Configuration Tag: mazes-v1


In [24]:
try:
    env.close()
except:
    pass
# Generate a seed for maze 
print("Generating new seed ...")
maze_seed = random.randint(1, 9999)
print("Loading environment ...")
# num = random.randint(0, 8)
# mission_file = run_config["env_config"]["mission_file"].replace("*", str(num))
# from glob import glob
# mission_files = glob(run_config["env_config"]["mission_file"])
# mission_file = random.choice(mission_files)
os.environ['MALMO_XSD_PATH'] = '/Users/gkos/Insync/Gdrive/Projects/UTK/Minecraft-AI/MalmoPlatform/Schemas'

env = MalmoMazeEnv(
            width=400,#env_config["width"],
            height=400,#env_config["height"],
            mazeseed=maze_seed,
            xml=env_config["mission_file"],
            millisec_per_tick=env_config['millisec_per_tick'],
            max_loop=c_general['max_loop'],
            mission_timeout_ms=c_general['mission_timeout_ms'],
            step_reward=c_general['step_reward'],
            win_reward=c_general['win_reward'],
            lose_reward=c_general['lose_reward'],
            action_space=c_general['action_space'],
            client_port=env_config['client_port'],
            time_wait=c_general['time_wait'])
print("Resetting environment ...")
_ = env.reset()
print("Environment Loaded!")
obs, reward, done, info = env.step(5)

Generating new seed ...
Loading environment ...
Resetting environment ...
Environment Loaded!


In [5]:
print("Possible actions and action space actions for current env:")
for action in range(6):
    print(f"Action: {action} -> {env.action_space[action]}")

Possible actions and action space actions for current env:
Action: 0 -> move 1
Action: 1 -> move -1
Action: 2 -> strafe 1
Action: 3 -> strafe -1
Action: 4 -> turn 1
Action: 5 -> turn -1


In [6]:
print("Sample an action")
action = env.action_space.sample()
print(f'{action} -> {env.action_space[action]}')

Sample an action
5 -> turn -1


In [7]:
# Take an action and get the observation, reward, done, and info
obs, reward, done, info = env.step(0)

In [12]:
print("Reward: ", reward)
print("Done: ", done)
print("Info: ", info)
print(f"Info.observations: len={len(info.observations)} "\
      f"type={type(info.observations[0])}")
print(f"Observation: type={type(obs)}, shape={obs.shape}")


Reward:  -1.0
Done:  False
Info:  WorldState (running): 102 obs, 1 rewards, 165 frames since last state.
Info.observations: len=1 type=<class 'malmo.MalmoPython.TimestampedString'>
Observation: type=<class 'numpy.ndarray'>, shape=(400, 400, 3)


In [29]:
# Find Usable Data
# Transform the info observation
import json
info_obs = json.loads(info.observations[-1].text)
floor_data = info_obs['floor10x10']
time_data = info_obs['TotalTime']
xpos_data = info_obs['XPos']
ypos_data = info_obs['YPos']
zpos_data = info_obs['ZPos']
yaw_data = info_obs['Yaw']  # where the player is facing
xp_data = info_obs['XP']
print("# ------ Usable data ------ #")
print(f"Var obs: a {obs.shape} np array")
print(f"Var info_obs['floor10x10']: a {len(floor_data)} list: {floor_data[:3]} ... ")
print(f"Var info_obs['TotalTime']: {time_data}")
print(f"Var info_obs['XPos']: {xpos_data}")
print(f"Var info_obs['YPos']: {ypos_data}")
print(f"Var info_obs['ZPos']: {zpos_data}")
print(f"Var info_obs['Yaw']: {yaw_data}")
print(f"Var info_obs['XP']: {xp_data}")
# print(obs)

# ------ Usable data ------ #
Var obs: a (400, 400, 3) np array
Var info_obs['floor10x10']: a 81 list: ['air', 'air', 'air'] ... 
Var info_obs['TotalTime']: 18196
Var info_obs['XPos']: 3.5
Var info_obs['YPos']: 227.0
Var info_obs['ZPos']: 1.5
Var info_obs['Yaw']: 270.0
Var info_obs['XP']: 0


In [ ]:
# Use the internal actions to move the agent
# move: walks forwards/backwards
#   0 -> Nothing, 1 -> Forward, -1 -> Backward
# strafe: walks left/right
#   0 -> Nothing, 1 -> Right, -1 -> Left
# turn: turns the camera left/right without moving
#   0 -> Nothing, 1 -> Right, -1 -> Left
env.agent_host.sendCommand("turn 1")

In [30]:
env.close()